# QSTK

This is my notebook to learn about QSTK or how to do quantitative financial modelling. 

The wiki can be found here. http://wiki.quantsoftware.org/index.php?title=QuantSoftware_ToolKit

## Reading historical data

In [ ]:
import QSTK.qstkutil.qsdateutil as du
import QSTK.qstkutil.tsutil as tsu
import QSTK.qstkutil.DataAccess as da

import matplotlib.pyplot as plt
import datetime as dt
import pandas as pd
import math
import numpy as np
%matplotlib inline

In [ ]:
ls_symbols = ["AAPL", "GLD", "GOOG", "$SPX", "XOM"]
dt_start = dt.datetime(2006,1,1)
dt_end = dt.datetime(2010, 12, 31)
dt_timeofday = dt.timedelta(hours=16)
ldt_timestamps = du.getNYSEdays(dt_start, dt_end, dt_timeofday)

In [ ]:
c_dataobj = da.DataAccess('Yahoo', cachestalltime=0)
ls_keys = ['open', 'high', 'low', 'close', 'volume', 'actual_close']
ldf_data = c_dataobj.get_data(ldt_timestamps, ls_symbols, ls_keys)
d_data = dict(zip(ls_keys, ldf_data))

Show me the open prices of AAPL

In [ ]:
d_data['open']['AAPL'].head()

In [ ]:
d_data['actual_close'].head()

In [ ]:
d_data['close'].head()

In [ ]:
na_price = d_data['close'].values
na_normalized_price = na_price / na_price[0, :]
plt.clf()
plt.plot(ldt_timestamps, na_normalized_price)
plt.legend(ls_symbols)
plt.ylabel('Adjusted Close')
plt.xlabel('Date')

## Daily Returns

It can be calculated using this formula

ret(t) = (price(t) / price(t-1)) - 1

In [ ]:
na_normalized_price

In [ ]:
na_rets = na_normalized_price.copy()
tsu.returnize0(na_rets)

Using pure pandas

In [ ]:
pct_change = pd.DataFrame(na_normalized_price).pct_change(1)
pct_change.head()

In [ ]:
daily_change_of_AAPL = pct_change.ix[:50, 0]

In [ ]:
plt.clf()
plt.plot(ldt_timestamps[0:51][:], daily_change_of_AAPL)
plt.legend(ls_symbols)
plt.ylabel('Adjusted Close')
plt.xlabel('Date')

The scatter plot of $SPX versus GLD

In [ ]:
plt.scatter(na_rets[:, 3], na_rets[:, 1], c='blue')

## Cumulative Daily Returns

daily_cum_ret(t) = daily_cum_ret(t-1) * (1 + daily_ret(t))

In [ ]:
cumulative_daily_returns = pct_change.cumsum()

In [ ]:
cumulative_daily_returns.head()

In [ ]:
cumulative_daily_returns.plot()

## Simulation

Going to write a function than returns, SD, avg_daily_returns, sharpe and cumulative return of portfolio

vol, daily_ret, sharpe, cum_ret = simulate(startdate, enddate, ['GOOG', 'AAPL', 'GLD', 'XOM'], [0.2, 0.3, 0.4, 0.1]

### Example

**Start Date**: January 1, 2011 <br>
**End Date**: December 31, 2011 <br>
**Symbols**: ['AAPL', 'GOOG', 'XOM', 'GLD'] <br>
**Optimal Allocations**: [0.4, 0.4, 0.0, 0.2] <br>
**Sharpe Ratio**: 1.02828403099 <br>
**Volatility (stdev of daily returns)**: 0.0101467067654 <br>
**Average Daily Return**: 0.000657261102001 <br>
**Cumulative Return**: 1.16487261965 <br>

## Sharpe Ratio
Sharpe = (sqrt(n) * avg(d)) / std(d)

In [ ]:
def simulate(dt_start, dt_end, symbols, allocs):
    dt_timeofday = dt.timedelta(hours=16)
    ldt_timestamps = du.getNYSEdays(dt_start, dt_end, dt_timeofday)

    c_dataobj = da.DataAccess('Yahoo', cachestalltime=0)
    ls_keys = ['open', 'high', 'low', 'close', 'volume', 'actual_close']
    ldf_data = c_dataobj.get_data(ldt_timestamps, symbols, ls_keys)
    d_data = dict(zip(ls_keys, ldf_data))

    na_price = d_data['close'].values
    na_normalized_price = na_price / na_price[0, :]
    na_port_value = np.sum(na_normalized_price * allocs, axis=1)
    
    na_rets = na_port_value.copy()
    daily_returns = tsu.returnize0(na_rets)
    
    avg = daily_returns.mean()
    std = daily_returns.std()
    sharpe_ratio = (math.sqrt(len(ldt_timestamps)) * avg) / std
    return float(std), float(avg), float(sharpe_ratio), allocs

In [ ]:
symbols = ['AAPL', 'GLD', 'GOOG', 'XOM']
dt_start = dt.datetime(2011, 1, 1)
dt_end = dt.datetime(2011, 12, 31)
allocs = [0.4, 0.4, 0.0, 0.2]
portfolio = simulate(dt_start, dt_end, symbols, allocs)
std, avg, sharpe_ratio, alloc = portfolio
print("Std: {}".format(std))
print("Average: {}".format(avg))
print("Sharpe ratio: {}".format(sharpe_ratio))

In [ ]:
symbols = ['AXP', 'HPQ', 'IBM', 'HNZ']
dt_start = dt.datetime(2010, 1, 1)
dt_end = dt.datetime(2010, 12, 31)
allocs = [0.0, 0.0, 0.0, 1.0]
portfolio = simulate(dt_start, dt_end, symbols, allocs)
std, avg, sharpe_ratio, alloc = portfolio
print("Std: {}".format(std))
print("Average: {}".format(avg))
print("Sharpe ratio: {}".format(sharpe_ratio))

## Simulation

In [ ]:
import itertools
perm_of_allocs = [[x /10.0 for x in t] for t in itertools.product(range(11), repeat=4) if sum(t)==10]

In [ ]:
possible_allocs = []
for i in range(10):
    for j in range(10):
        for k in range(10):
            for l in range(10):
                if (i + j + k + l == 10):
                    possible_allocs.append([i/10.0, j/10.0, k/10.0, l/10.0])

In [ ]:
symbols = ['AAPL', 'GOOG', 'IBM', 'MSFT']
dt_start = dt.datetime(2011, 1, 1)
dt_end = dt.datetime(2011, 12, 31)

portfolios = []
for alloc in possible_allocs:
    portfolios.append(simulate(dt_start, dt_end, symbols, alloc))
        
portfolios.sort(key = lambda tup: tup[2], reverse=True)
portfolios[:1]

In [ ]:
symbols = ['BRCM', 'ADBE', 'AMD', 'ADI']
dt_start = dt.datetime(2010, 1, 1)
dt_end = dt.datetime(2010, 12, 31)

portfolios = []
for alloc in possible_allocs:
    portfolios.append(simulate(dt_start, dt_end, symbols, alloc))

portfolios.sort(key = lambda tup: tup[2], reverse=True)
portfolios[:1]

In [ ]:
symbols = ['BRCM', 'TXN', 'AMD', 'ADI']
dt_start = dt.datetime(2011, 1, 1)
dt_end = dt.datetime(2011, 12, 31)

portfolios = []
for alloc in possible_allocs:
    portfolios.append(simulate(dt_start, dt_end, symbols, alloc))
    
portfolios.sort(key = lambda tup: tup[2], reverse=True)
portfolios[:1]

In [ ]:
symbols = ['BRCM', 'TXN', 'IBM', 'HNZ']
dt_start = dt.datetime(2010, 1, 1)
dt_end = dt.datetime(2010, 12, 31)

portfolios = []
for alloc in possible_allocs:
    portfolios.append(simulate(dt_start, dt_end, symbols, alloc))
    
portfolios.sort(key = lambda tup: tup[2], reverse=True)
portfolios[:1]

In [ ]:
symbols = ['C', 'GS', 'IBM', 'HNZ']
dt_start = dt.datetime(2010, 1, 1)
dt_end = dt.datetime(2010, 12, 31)

portfolios = []
for alloc in possible_allocs:
    portfolios.append(simulate(dt_start, dt_end, symbols, alloc))
    
portfolios.sort(key = lambda tup: tup[2], reverse=True)
portfolios[:1]